In [1]:
# ! pip install xgboost

In [2]:
# Import basic libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Matplotlib is building the font cache; this may take a moment.


In [3]:
df = pd.read_csv('Dataset/phishing_site_urls.csv')

In [4]:
df.head()

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad


In [5]:
df.shape

(549346, 2)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549346 entries, 0 to 549345
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   URL     549346 non-null  object
 1   Label   549346 non-null  object
dtypes: object(2)
memory usage: 8.4+ MB


In [7]:
df.isnull().sum()

URL      0
Label    0
dtype: int64

In [8]:
df.Label.value_counts()

Label
good    392924
bad     156422
Name: count, dtype: int64

In [9]:
import re

In [10]:
def tokenize_url(url):
    return re.split(r'\W+', url)

In [11]:
df['tokens'] = df['URL'].apply(tokenize_url)

In [13]:
%pip install gensim

from gensim.models import Word2Vec


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
    --------------------------------------- 0.5/24.4 MB 3.4 MB/s eta 0:00:08
   -- ------------------------------------- 1.3/24.4 MB 3.7 MB/s eta 0:00:07
   --- ------------------------------------ 2.4/24.4 MB 4.1 MB/s eta 0:00:06
   ----- ---------------------------------- 3.1/24.4 MB 4.2 MB/s eta 0:00:06
   ------ --------------------------------- 4.2/24.4 MB 4.4 MB/s eta 0:00:05
   --------- ------------------------------ 5.5/24.4 MB 4.7 MB/s eta 0:00:04
   ----------- ---------------------------- 7.1/24.4 MB 5.1 MB/s eta 0:00:04
   -------------- ------------------------- 8.7/24.4 MB 5.5 MB/s eta 0:00:03
   ---------------- ----------------------- 10.2/24.4 MB 5.7 MB/s eta 0:00:03
   ------------------ --------------------- 11.0/24.4 MB 5.7 MB/s eta 0:00:03
   ------------------ --------------------- 11.5/24.4 MB 5.3 MB/s eta 0:00:03
   -------------------- ------------------- 12.3/24.4 MB 5.1 MB/s eta 0:00:03
 

In [14]:
# Train Word2Vec on tokenized URLs
w2v_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

In [15]:
# Create URL embedding by averaging token vectors
def get_embedding(tokens):
    valid_tokens = [t for t in tokens if t in w2v_model.wv]
    if not valid_tokens:
        return np.zeros(w2v_model.vector_size)
    return np.mean([w2v_model.wv[t] for t in valid_tokens], axis=0)

In [16]:
df['embedding'] = df['tokens'].apply(get_embedding)

In [17]:
x = np.vstack(df['embedding'].values)
y = df['Label'].values

In [18]:
df.head()

,URL,Label,tokens,embedding
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,"[nobell, it, 70ffb52d079109dca5664cce6f3173737...","[-0.34559786, -0.8200375, -1.0713806, -1.42736..."
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,"[www, dghjdgf, com, paypal, co, uk, cycgi, bin...","[-0.11090333, -0.7987742, -0.47629133, -1.5343..."
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,"[serviciosbys, com, paypal, cgi, bin, get, int...","[-0.61831033, -1.0708354, -1.034574, -1.778311..."
3,mail.printakid.com/www.online.americanexpress....,bad,"[mail, printakid, com, www, online, americanex...","[0.023933887, -0.8406321, -1.5115973, -1.46645..."
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,"[thewhiskeydregs, com, wp, content, themes, wi...","[0.3377234, -1.9654123, -0.42500973, -0.931080..."


### Train traditional Machine learning algorithms

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
# Train/test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [22]:
%pip install xgboost

import xgboost as xgb


   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   - -------------------------------------- 3.4/72.0 MB 20.2 MB/s eta 0:00:04
   -- ------------------------------------- 4.2/72.0 MB 10.9 MB/s eta 0:00:07
   -- ------------------------------------- 4.7/72.0 MB 8.4 MB/s eta 0:00:09
   -- ------------------------------------- 5.2/72.0 MB 6.5 MB/s eta 0:00:11
   --- ------------------------------------ 5.8/72.0 MB 5.6 MB/s eta 0:00:12
   --- ------------------------------------ 6.6/72.0 MB 5.3 MB/s eta 0:00:13
   ---- ----------------------------------- 7.6/72.0 MB 5.2 MB/s eta 0:00:13
   ---- ----------------------------------- 8.9/72.0 MB 5.3 MB/s eta 0:00:12
   ----- ---------------------------------- 10.2/72.0 MB 5.5 MB/s eta 0:00:12
   ------ --------------------------------- 11.5/72.0 MB 5.6 MB/s eta 0:00:11
   ------ --------------------------------- 12.6/72.0 MB 5.4 MB/s eta 0:00:11
   ------- -------------------------------- 13.4/72.0 MB 5.3 MB/s eta 0:00:12


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Train XGBoost
model_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

In [ ]:
# model_xgb.fit(X_train, y_train)

### LLM Model Training

In [ ]:
data = df[['URL', 'Label']]

In [ ]:
data.head()

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad


In [ ]:
data['Label'] = data['Label'].map({'good': 0, 'bad': 1})

In [ ]:
data.rename(columns={'Label': 'label'}, inplace=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd

In [ ]:
# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(data)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize(example):
    return tokenizer(example['URL'], truncation=True, padding='max_length')

In [ ]:
dataset = dataset.map(tokenize, batched=True)
dataset = dataset.train_test_split(test_size=0.01)

Map:   0%|          | 0/549346 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
# Save model
model.save_pretrained("phishing_model")
tokenizer.save_pretrained("phishing_model")

In [ ]:
# Load model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("phishing_model")
tokenizer = AutoTokenizer.from_pretrained("phishing_model")

In [ ]:
# prediction
import torch

urls = ["http://phishy.site", "https://secure.bank.com"]
inputs = tokenizer(urls, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
predictions = torch.argmax(logits, dim=1)
print(predictions)  # tensor([1, 0]) → bad, good

In [ ]:
# Get label back
id2label = {0: 'good', 1: 'bad'}
pred_labels = [id2label[int(pred)] for pred in predictions]
print(pred_labels)